# **DetectGPT for dummies**: Identifying AI-generated text
This notebook implements the **DetectGPT** method from Mitchell et al. (2023) [1], which helps determine whether a given text is AI-generated. The approach involves perturbing the text and analyzing its log-probabilities.

In [4]:
import os
import re
import json
import torch
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from functools import lru_cache
from transformers import AutoModelForCausalLM, AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## I- **Model setup**

This part is a simple setup of different transformer based models that will be needed to:
1. produce the AI-generated text - ``generation_model``
2. compute the log-probablities - ``computation_model``
3. perturb the text with the T5 perturbation - ``t5_model``

### 1. **Text generation**

In [5]:
CACHE_DIR = "/tmp/huggingface"

GENERATION_MODEL_NAME = "gpt2"
# Model list (all tested)
# gpt2
# gpt2-large
# EleutherAI/gpt-j-6B
# EleutherAI/gpt-neox-20b

TORCH_DTYPE = torch.bfloat16 # use bfloat16 for all models

# Load model
generation_model = AutoModelForCausalLM.from_pretrained(GENERATION_MODEL_NAME, torch_dtype=TORCH_DTYPE, cache_dir=CACHE_DIR)

# Load tokenizer 
generation_tokenizer = AutoTokenizer.from_pretrained(GENERATION_MODEL_NAME)

# Set model to evaluation mode
generation_model.eval()

generation_model.to(device)
print(device)

c:\Users\paulh\miniconda3\envs\detectgpt_env\lib\site-packages\huggingface_hub-0.28.1-py3.8.egg\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


cpu


### 2. **Computation**

In [6]:
CACHE_DIR = "/tmp/huggingface"
COMPUTATION_MODEL_NAME = "openai-community/gpt2-large"
TORCH_DTYPE = torch.bfloat16 # use bfloat16 for all models

# Load model
computation_model = AutoModelForCausalLM.from_pretrained(COMPUTATION_MODEL_NAME, torch_dtype=TORCH_DTYPE, cache_dir=CACHE_DIR)

# Load tokenizer 
computation_tokenizer = AutoTokenizer.from_pretrained(COMPUTATION_MODEL_NAME, cache_dir=CACHE_DIR)
computation_tokenizer.pad_token = computation_tokenizer.eos_token

# Set model to evaluation mode (ensures stable log prob estimation + disables dropout)
computation_model.eval()

computation_model.to(device)
print(device)

cpu


### 3. **Perturbation**

In [7]:
CACHE_DIR = "/tmp/huggingface"
PERTURBATION_MODEL_NAME = "t5-large"
TORCH_DTYPE = torch.bfloat16 # use bfloat16 for all models

# Load model
t5_model = T5ForConditionalGeneration.from_pretrained(PERTURBATION_MODEL_NAME, torch_dtype=TORCH_DTYPE, cache_dir=CACHE_DIR)

# Load tokenizer 
t5_tokenizer = T5Tokenizer.from_pretrained(PERTURBATION_MODEL_NAME, cache_dir=CACHE_DIR)

# Set to evaluation mode
t5_model.eval()

t5_model.to(device)
print(device)

c:\Users\paulh\miniconda3\envs\detectgpt_env\lib\site-packages\transformers\models\t5\tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be 

cpu


## II- **Code setup**

### 1. **🔀 Text perturbation**

This section defines the **T5-based perturbation function**, which modifies the input text slightly while preserving its meaning. 

- **Why is perturbation needed?** AI-generated text often sits in **low-curvature** probability regions, meaning slight perturbations can significantly change their log probabilities
- **How does it work?** The **T5 model** introduces variations to the text and helps in detecting AI-generated content

These perturbed texts will later be compared to their original versions to compute the discrepancy scores d.

In [8]:
def batch_mask_text(texts, mask_ratio=0.15, max_words=370):
    """Mask multiple texts at once."""
    masked_texts = []
    
    for text in texts:
        words = text.split()
        
        # Truncate text
        if len(words) > max_words:
            words = words[:max_words]
        
        num_masks = int(len(words) * mask_ratio)
        
        # Randomly select spans to mask (sorted in reverse to avoid index shifts)
        mask_indices = sorted(random.sample(range(len(words) - 1), num_masks), reverse=True)
        
        for i, idx in enumerate(mask_indices):
            words[idx] = f"<extra_id_{i}>"
            if idx + 1 < len(words):  # Ensure a 2-word span
                del words[idx + 1]  # Remove instead of replacing with ""
        
        masked_texts.append(" ".join(words))
    
    return masked_texts

def batch_replace_masks(texts, batch_size=128):
    """Generate T5 model outputs for masked texts in batches."""
    all_outputs = []
    
    # Process in batches
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        n_expected = [text.count("<extra_id_") for text in batch_texts]
        stop_id = t5_tokenizer.encode(f"<extra_id_{max(n_expected)}>")[0]
        
        tokens = t5_tokenizer(batch_texts, return_tensors="pt", padding=True)
        
        # Move input tensors to model's device
        with torch.no_grad():
            outputs = t5_model.generate(
                input_ids=tokens["input_ids"].to(t5_model.device),
                attention_mask=tokens["attention_mask"].to(t5_model.device),
                max_length=150,
                do_sample=True,
                top_p=0.9,
                num_return_sequences=1,
                eos_token_id=stop_id
            )
            
        # Move outputs back to CPU to save GPU memory
        outputs = outputs.detach().cpu()
        batch_decoded = t5_tokenizer.batch_decode(outputs, skip_special_tokens=False)
        all_outputs.extend(batch_decoded)
    
    return all_outputs

def batch_extract_fills(texts):
    """Extract the generated fills from T5's output for multiple texts."""
    extracted_fills = []
    for text in texts:
        text = text.replace("<pad>", "").replace("</s>", "").strip()
        
        # Use regex to extract text inside <extra_id_X> tokens
        fills = re.findall(r"<extra_id_\d+>\s*(.*?)\s*(?=<extra_id_\d+>|$)", text)
        
        # Clean extracted tokens
        extracted_fills.append([fill.strip() for fill in fills])
    
    return extracted_fills

def batch_apply_extracted_fills(masked_texts, extracted_fills):
    """Replace mask tokens in the masked texts with generated fills."""
    filled_texts = []
    
    for masked_text, fills in zip(masked_texts, extracted_fills):
        if not fills:
            filled_texts.append(masked_text)
            continue
        
        filled_text = masked_text
        # Iterate through expected mask positions and replace them
        for i, fill in enumerate(fills):
            filled_text = filled_text.replace(f"<extra_id_{i}>", fill, 1)
        
        filled_texts.append(filled_text)
    
    return filled_texts

In [9]:
def t5_perturbation(text: str, batch_size: int) -> str:
    """
    T5 perturbation - batch version

    Args:
        text (str): the input texts to be perturbed
        batch_size (int): batch_size for compute

    Returns:
        all_perturbed_texts (str): the perturbed texts
    """
    # Step 1: mask all texts at once
    all_masked_texts = batch_mask_text(text)

    # Step 2: generate replacements in batches
    all_raw_fills = batch_replace_masks(all_masked_texts, batch_size)

    # Step 3: extract fills
    all_extracted_fills = batch_extract_fills(all_raw_fills)

    # Step 4: apply fills
    all_perturbed_texts = batch_apply_extracted_fills(all_masked_texts, all_extracted_fills)
    
    return all_perturbed_texts

### 2. **🔍 Main functions: *DetectGPT* Method**

This section implements the **DetectGPT method**.

- **Key idea:** once again, AI-generated texts often **reside in low-curvature probability regions**.
- **How does it work?**
  - We perturb the text multiple times (``num_perturbation``). We will use ``n_samples`` texts with ``max_length`` words.
  - Compute log probabilities for both **original** and **perturbed** texts
  - Measure the **discrepancy score** (a higher score suggests AI-generated text)

In [10]:
def batch_average_log_prob(texts, batch_size=128):
    """Calculate average log probability for multiple texts in batches."""
    
    all_log_probs = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        # Tokenize input
        inputs = computation_tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        with torch.no_grad():
            outputs = computation_model(input_ids, labels=input_ids, attention_mask=attention_mask)

        # Extract logits
        logits = outputs.logits  # [batch_size, seq_len, vocab_size]

        # Shift logits and labels to align
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = input_ids[..., 1:].contiguous()
        shift_mask = attention_mask[..., 1:].contiguous()  # Ensure mask aligns

        # Compute per-token loss
        loss_fct = torch.nn.CrossEntropyLoss(reduction='none', ignore_index=computation_tokenizer.pad_token_id)
        loss_per_token = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        # Reshape to [batch_size, seq_length - 1]
        loss_per_token = loss_per_token.view(shift_labels.size())

        # Compute per-sample log prob
        sample_losses = []
        for j in range(loss_per_token.size(0)):
            mask = shift_mask[j].bool()  # Use shift_mask for actual tokens
            if mask.sum() > 0:
                sample_loss = loss_per_token[j][mask].mean().item()
                sample_losses.append(-sample_loss)  # Negative loss as log prob
            else:
                sample_losses.append(float('-inf'))  # Avoid zero prob bias

        all_log_probs.extend(sample_losses)

    return all_log_probs

In [ ]:
# Main optimized processing loop
def optimized_processing(data, num_samples=300, max_length=50, num_perturbation=100, batch_size=128):
    log_probs_per_text_transformed = []
    
    # Process original texts in batches
    original_texts = [" ".join(data[j]["text"].split()[:max_length]) for j in range(num_samples)]
    log_probs_per_text_base = batch_average_log_prob(original_texts, batch_size)
    
    # Inside the loop in optimized_processing()
    for perturbation_idx in tqdm(range(num_perturbation), desc=f"Processing {num_perturbation} perturbations for {num_samples} texts. Perturbation number:"):
        all_perturbed_texts = t5_perturbation(original_texts,batch_size)
        all_log_probs = batch_average_log_prob(all_perturbed_texts, batch_size)
        
        # Organize results by original text
        for j in range(num_samples):
            if perturbation_idx == 0:
                log_probs_per_text_transformed.append([])
            log_probs_per_text_transformed[j].append(all_log_probs[j])
    
    return log_probs_per_text_base, log_probs_per_text_transformed

In [ ]:
def gen_perturbed(data, num_samples=300, max_length=50, num_perturbation=100, batch_size=128):
    '''Generates pertubations for text. Returns list of length num_pertubations, each entry being a JSON object with perturbed text'''
    # Initialise list to store all perturbed JSON
    all_perturbed_texts = []
    
    # Process original texts in batches
    original_texts = [" ".join(data[j]["text"].split()[:max_length]) for j in range(num_samples)]
    
    # Iterate for length num_pertubation
    for perturbation_idx in tqdm(range(num_perturbation), desc=f"Processing {num_perturbation} perturbations for {num_samples} texts. Perturbation number:"):

        # Randomly select 15% of text to mask, creates pertubations
        perturbed_texts = t5_perturbation(original_texts,batch_size)

        # Store in list
        all_perturbed_texts.append(perturbed_texts)
    
    return all_perturbed_texts

In [ ]:
def compare_log_prob(original_texts, all_perturbed_texts, num_perturbation=100, batch_size=128):
    '''Compares log probs of original text vs list of JSONs perturbed texts'''
    # Initialise list to store log prob of each perturbed JSON
    log_probs_per_text_pert = []

    # Get num_samples
    num_samples = len(original_texts)

    # Calculate log prob for original text in batch
    log_probs_per_text_base = batch_average_log_prob(original_texts, batch_size)

    # Iterate 
    for perturbation in tqdm(range(num_perturbation)):

        # Get the JSON file
        perturbed_texts = all_perturbed_texts[perturbation]

        # Calculate log prob
        log_probs_per_text_tran = batch_average_log_prob(perturbed_texts, batch_size)

        # Organize results by original text
        for j in range(num_samples):
            if perturbation == 0:
                log_probs_per_text_pert.append([])
            log_probs_per_text_pert[j].append(log_probs_per_text_tran[j])
        
    return log_probs_per_text_base, log_probs_per_text_pert

In [12]:
def compute_detectgpt_discrepancy(log_probs_per_text_base, log_probs_per_text_transformed):
    """
    Compute the DetectGPT discrepancy metric for each of the n_samples texts
    Calculated for num_perturbations perturbations

    Args:
        log_probs_per_text_base (list): original log probability of each text
        log_probs_per_text_transformed (list): list of size n_samples where each element is a list of the num_perturbations perturbed log probabilities

    Returns:
        discrepancy_scores (list): list of discrepancy values (d) for the n_samples texs
    """
    num_samples = len(log_probs_per_text_base) 
    discrepancy_scores = []

    for i in range(num_samples):
        original_log_prob = log_probs_per_text_base[i]
        perturbed_log_probs = log_probs_per_text_transformed[i] # List of perturbed log probs
        num_perturbations = len(perturbed_log_probs) # Number of perturbations

        # Compute mean log probability of the perturbed texts
        mu = sum(perturbed_log_probs) / num_perturbations  

        # Compute discrepancy
        d = original_log_prob - mu  
        discrepancy_scores.append(d)
    
    return discrepancy_scores

### 3. **Utility functions**

In [13]:
# Memory management utilities
def clear_cuda_cache():
    """Clear CUDA cache to free up memory."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


# Add caching for tokenization
@lru_cache(maxsize=1024)
def cached_tokenize(text, is_t5=False):
    """Cache tokenization results to avoid repeated work."""
    if is_t5:
        return t5_tokenizer(text, return_tensors="pt", padding=True)
    else:
        return computation_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

## III- **Data loading**

**📌 Dataset format guidelines**

All datasets (human-written and AI-generated) must follow this format:

- Stored as a **`.jsonl`** where each line is a dictionary.
- Each entry contains (minimum requirement):
  - `"text"`: the text content
  - `"model"`: for human text please label it as `"human"` and for AI-generated texts, please specify the model used (e.g. ``"gpt2-large"``)
  - `"source"`: the origin of the text (e.g., `"wikihow"`, `"reddit"`, `"news articles"`)

#### Exemple (as in ``subtaskB_train.jsonl`` located in `Datasets\SemEval2024-Task8`):
```json
{"text": "A groundbreaking discovery in physics was made today.", "model": "human", "source": "news articles"}
{"text": "The AI revolution is shaping the future of work.", "model": "chatGPT", "source": "AI Generated"}


### 1. **Human texts**

In [28]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
FILE_RELATIVE_PATH = "Datasets\SemEval2024-Task8\subtaskB_train.jsonl"
FILE_PATH = os.path.join(BASE_DIR,FILE_RELATIVE_PATH) 

if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"File not found: {FILE_PATH}")

data_human = []

# Efficiently process the file line by line
with open(FILE_PATH, "r", encoding="utf-8") as file:
    for line in file:
        record = json.loads(line)  # Parse JSON once
        if record.get("model") == "human":
            data_human.append(record)

# Print first human record
print("First human text record:", data_human[0] if data_human else "No human data found.")

First human text record: {'text': ' It will work with the Forza Motorsport disc in. Simply it will load but this problem will occur if your disc is dirty or damaged. Clean it with water on the back of the disc. There can be scratches because discs are fragile and can break easily. So always clean it with a soft, lint-free cloth.;\n, If you have the game that includes the bonus feature, Xbox Live Arcade, You will get the dashboard menu that has "Forza Motorsport" and "Xbox Live Arcade". Choose Xbox Live Arcade if you want to play some arcade games, If you want to play Forza Motorsport, Simply press "A" on the Forza Motorsport Feature.\n\n\nThe Main Menu features one of those choices after loading or creating a profile on the game.\n\nArcade Race: This is simply the quick race mode. You can race at a circuit, sprint, or a very interesting race. Just simply choose a class and choose a car and then choose a race and the race will load, Also the cars are featured in multiplayer, career, and

### 2. **AI-generated texts**

#### 2.1. *Option 1: produce own AI-generated texts*

In [17]:
def generate_text(prompt: str, max_length: int) -> str:
    """
    Generate AI text from a given prompt

    Args:
        prompt (str): prompt to generate text
        max_length (int): max length of generated text

    Returns:
        cleaned_text (str): cleaned generated text
    """
    inputs = generation_tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        output = generation_model.generate(**inputs, max_length=max_length, do_sample=True, temperature=0.7)
    
    generated_text = generation_tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

def generate_dataset(N: int, max_length: int, generation_model_name: str) -> list:
    """
    Generates a dataset of N AI-generated texts in the required dictionary format.

    Args:
        N (int): Number of AI-generated texts
        max_length (int): Maximum length of each generated text
        generation_model_name (str): Name of the AI generation model

    Returns:
        data_ai (list): Dataset of AI-generated texts (list of dictionaries)
    """
    prompt = "In a faraway galaxy,"
    data_ai = [
        {
            "text": generate_text(prompt, max_length),
            "model": generation_model_name,
            "source": "FleLLM"
        }
        for _ in range(N)
    ]
    return data_ai

In [18]:
N = 10
max_length = 100
data_ai_generated = generate_dataset(N, max_length, GENERATION_MODEL_NAME)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [19]:
# Saving the dataset in the correct .jsonl format

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
FILE_RELATIVE_PATH = "Datasets\AI-generated\dataset_ai.jsonl"
FILE_PATH = os.path.join(BASE_DIR,FILE_RELATIVE_PATH)

with open(FILE_PATH, "w", encoding="utf-8") as f:
    for entry in data_ai_generated:
        f.write(json.dumps(entry) + "\n")

In [20]:
# Print first AI-generated text record
print("First AI-generated text record:", data_ai_generated[0] if data_ai_generated else "No AI-generated data found.")

First AI-generated text record: {'text': 'In a faraway galaxy, the alien galaxy is far away, and on the surface of that distant galaxy, we see the stars. The galaxies are very dense, and this is a big difference.\n\nBut there are many other galaxies, and they are very dense, and we can see some of them. There are three other galaxies, and they are very dense, and they are very dark. We can look at these stars as if they are the equivalent of a telescope. We can look', 'model': 'gpt2', 'source': 'FleLLM'}


#### 2.2. *Option 2: load AI-generated texts from a dataset*

In [29]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
FILE_RELATIVE_PATH = "Datasets\SemEval2024-Task8\subtaskB_train.jsonl"
# FILE_RELATIVE_PATH = "Datasets\AI-generated\dataset_ai.jsonl"
FILE_PATH = os.path.join(BASE_DIR,FILE_RELATIVE_PATH) 

if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"File not found: {FILE_PATH}")

data_ai_dataset = []

# Read entire file and parse as JSON list
with open(FILE_PATH, "r", encoding="utf-8") as file:
    for line in file:
        record = json.loads(line)  # Parse JSON once
        if record.get("model") != "human":
            data_ai_dataset.append(record)

# Print first AI-generated text record
print("First AI-generated text record:", data_ai_dataset[0] if data_ai_dataset else "No AI-generated data found.")

First AI-generated text record: {'text': 'Forza Motorsport is a popular racing game that provides players with the ability to race on various tracks and in different vehicles. Whether you\'re a seasoned racer or a newbie, playing Forza Motorsport can be a fun experience. In this article, we will take you through the different steps on how to play Forza Motorsport.\n\nStep 1. Insert The Game Disc\n\nThe first step is to insert the game disc into your console or computer. Follow the instructions to set up the game.\n\nStep 2. Choose Your Game\n\nOnce the game is set up, choose the game you\'d like to play. Forza Motorsport has different modes: Career, Free Play, and Arcade. In this article, we will focus on the Arcade mode.\n\nStep 3. Just Make A Quick Race By The Arcade Mode\n\nOnce the Arcade mode is selected, choose "Quick Race" to get started quickly.\n\nStep 4. Pick A Racetrack\n\nPick a racetrack from the different ones available like Road Atlanta, New York, Rio de Janeiro, Maple V

## IV- **Exemple usage**

Human texts

In [ ]:
DATA = data_human
NUM_SAMPLES = 300
MAX_LENGTH = 50
NUM_PERTURBATIONS = 100
BATCH_SIZE = 128

log_probs_base_human, log_probs_transformed_human = optimized_processing(DATA, NUM_SAMPLES, MAX_LENGTH, NUM_PERTURBATIONS, BATCH_SIZE)
discrepancy_scores_human = compute_detectgpt_discrepancy(log_probs_base_human, log_probs_transformed_human)

results_human = {}
results_human["log_probs_base"] = log_probs_base_human
results_human["log_probs_transformed"] = log_probs_transformed_human
results_human["discrepancy_scores"] = discrepancy_scores_human

In [ ]:
# Saving results
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
FILE_RELATIVE_PATH = "Results\experiment_0_results_human.json"
FILE_PATH = os.path.join(BASE_DIR,FILE_RELATIVE_PATH) 
with open(FILE_PATH, "w") as f:
    json.dump(results_human, f, indent=2)

AI-generated texts

In [ ]:
DATA = data_ai_generated # or DATA = data_ai_dataset
NUM_SAMPLES = 300
MAX_LENGTH = 50
NUM_PERTURBATIONS = 100
BATCH_SIZE = 128

log_probs_base_ai, log_probs_transformed_ai = optimized_processing(DATA, NUM_SAMPLES, MAX_LENGTH, NUM_PERTURBATIONS, BATCH_SIZE)
discrepancy_scores_ai = compute_detectgpt_discrepancy(log_probs_base_ai, log_probs_transformed_ai)

results_ai = {}
results_ai["log_probs_base"] = log_probs_base_ai
results_ai["log_probs_transformed"] = log_probs_transformed_ai
results_ai["discrepancy_scores"] = discrepancy_scores_ai

In [ ]:
# Saving results
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
FILE_RELATIVE_PATH = "Results\experiment_0_results_ai.json"
FILE_PATH = os.path.join(BASE_DIR,FILE_RELATIVE_PATH) 

with open(FILE_PATH, "w") as f:
    json.dump(results_ai, f, indent=2)

## V- **Analysis**

Do whatever you want

## References
[1] E. Mitchell, C. Lin, A. Bosselut, and C. D. Manning, "DetectGPT: Zero-Shot Machine-Generated Text Detection using Probability Curvature" *arXiv preprint*, 2023. Available at: [arXiv:2301.11305](https://arxiv.org/abs/2301.11305)